In [96]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pandas as pd
import re


In [97]:

# GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cuda


In [98]:

# T5 모델과 토크나이저 로드
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# GPU 메모리 확인
if device == 'cuda':
    print(torch.cuda.memory_summary(device=device))

# 리뷰 데이터를 읽어옴
file_path = '../외국음식전문점_테스트.csv'
data = pd.read_csv(file_path)

# 데이터의 첫 1000개 행만 사용
reducedData = data.iloc[:1000, :]

# 결측치 또는 NaN 데이터를 처리하여, NaN 값을 빈 문자열로 대체
reducedData['content'] = reducedData['content'].fillna('')

# 데이터 전처리: 불필요한 공백, 특수문자 제거
def clean_text(text):
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z0-9., ]', '', text)
    return text

# 모든 리뷰 텍스트 전처리
reducedData['clean_content'] = reducedData['content'].apply(clean_text)


|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1347 MiB |  15653 MiB |   3277 GiB |   3276 GiB |
|       from large pool |    987 MiB |  15473 MiB |   2817 GiB |   2816 GiB |
|       from small pool |    360 MiB |    519 MiB |    459 GiB |    459 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   1347 MiB |  15653 MiB |   3277 GiB |   3276 GiB |
|       from large pool |    987 MiB |  15473 MiB |   2817 GiB |

C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\625024579.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['content'] = reducedData['content'].fillna('')
C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\625024579.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['clean_content'] = reducedData['content'].apply(clean_text)


In [99]:

# 데이터셋 준비
def preprocess_data(texts, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt"
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# 마스킹 함수
def create_mask(input_ids):
    return (input_ids != tokenizer.pad_token_id).float()

# 전처리된 데이터로부터 입력과 마스크 생성
train_inputs, train_masks = preprocess_data(reducedData['clean_content'])
train_labels, _ = preprocess_data(reducedData['content'])

# 배치 처리 및 데이터셋 준비
batch_size = 4  # 배치 크기 설정
accumulation_steps = 4  # 그래디언트 축적 단계
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 혼합 정밀도 학습을 위한 스케일러 설정
scaler = GradScaler()

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)


C:\Users\G-01\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [101]:

# 학습 루프
num_epochs = 3  # 에포크 수 설정
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(train_dataloader)):
        batch_inputs, batch_masks, batch_labels = [b.to(device) for b in batch]
        
        with autocast():  # 혼합 정밀도 학습 사용
            outputs = model(input_ids=batch_inputs, attention_mask=batch_masks, labels=batch_labels)
            loss = outputs.loss / accumulation_steps  # 그래디언트 축적을 위해 손실 나누기

        scaler.scale(loss).backward()  # 스케일된 그래디언트 적용

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)  # 옵티마이저 업데이트
            scaler.update()  # 스케일러 업데이트
            optimizer.zero_grad()  # 그래디언트 초기화

        total_loss += loss.item()

    print(f"Loss after epoch {epoch + 1}: {total_loss / len(train_dataloader)}")


Epoch 1


100%|██████████| 250/250 [14:27<00:00,  3.47s/it]


Loss after epoch 1: 1.291033679485321
Epoch 2


100%|██████████| 250/250 [14:34<00:00,  3.50s/it]


Loss after epoch 2: 0.12410829064249992
Epoch 3


100%|██████████| 250/250 [15:02<00:00,  3.61s/it]

Loss after epoch 3: 0.058464961193501946


In [102]:

# 파인튜닝이 끝난 후 모델 저장
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")

# 요약문 생성 함수
def generate_summary(review):
    input_ids = tokenizer.encode(f"summarize: {review}", return_tensors="pt").to(device)
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 생성된 요약 확인
for i in range(5):  # 5개 리뷰만 샘플로 확인
    print(f"Original: {reducedData['content'].iloc[i]}")
    print(f"Summary: {generate_summary(reducedData['content'].iloc[i])}")


Original: 언제나줄서야먹을수있는테이블좀 늘려주세요 아님확장
Summary: 
Original: 목동에서 젤좋아하는 곳인데 오늘 고기도 여러점인데도  너무질겨서 하나도 못먹었어요.국물도 다른날이랑 달랐어요.몇젓갈안먹고 그냥왔어요.
Summary: 
Original: ㆍ드렁킨타이 (태국음식전문)ㆍ소고기쌀국수  10,500팟타이ㆍ똠양꿍ㆍ쏨땀 등등
Summary: 
Original: 와~~맛있고, 양이 넉넉합니다.테이블이 많지 않지만 타이밍 잘 맞으면 대기 없이 먹을 수 있어요.
Summary: 
Original: 친구동네 갔다 식사하러 들렀는데 완전 맛집이었네요. 고소한 팟타이 와 시원한 쌀국수가 취향저격이네요
Summary: .


In [104]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pandas as pd
import re

# GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# T5 모델과 토크나이저 로드
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# 리뷰 데이터를 읽어옴
file_path = '../외국음식전문점_테스트.csv'
data = pd.read_csv(file_path)

# 데이터의 첫 1000개 행만 사용
reducedData = data.iloc[:3000, :]

# 결측치 또는 NaN 데이터를 처리하여, NaN 값을 빈 문자열로 대체
reducedData['content'] = reducedData['content'].fillna('')

# 데이터 전처리: 불필요한 공백, 특수문자 제거
def clean_text(text):
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z., ]', '', text)
    return text

# 모든 리뷰 텍스트 전처리
reducedData['clean_content'] = reducedData['content'].apply(clean_text)

# 데이터셋 준비
def preprocess_data(texts, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt"
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# 마스킹 함수
def create_mask(input_ids):
    return (input_ids != tokenizer.pad_token_id).float()

# 전처리된 데이터로부터 입력과 마스크 생성
train_inputs, train_masks = preprocess_data(reducedData['clean_content'])
train_labels, _ = preprocess_data(reducedData['content'])

# 배치 처리 및 데이터셋 준비
batch_size = 4  # 배치 크기 설정
accumulation_steps = 4  # 그래디언트 축적 단계
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 혼합 정밀도 학습을 위한 스케일러 설정
scaler = GradScaler()

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프
num_epochs = 1  # 에포크 수 설정
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(train_dataloader)):
        batch_inputs, batch_masks, batch_labels = [b.to(device) for b in batch]
        
        with autocast():  # 혼합 정밀도 학습 사용
            outputs = model(input_ids=batch_inputs, attention_mask=batch_masks, labels=batch_labels)
            loss = outputs.loss / accumulation_steps  # 그래디언트 축적을 위해 손실 나누기

        scaler.scale(loss).backward()  # 스케일된 그래디언트 적용

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)  # 옵티마이저 업데이트
            scaler.update()  # 스케일러 업데이트
            optimizer.zero_grad()  # 그래디언트 초기화

        total_loss += loss.item()

    print(f"Loss after epoch {epoch + 1}: {total_loss / len(train_dataloader)}")

# 파인튜닝이 끝난 후 모델 저장
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")

# 요약문 생성 함수
def generate_summary(review):
    input_ids = tokenizer.encode(f"summarize: {review}", return_tensors="pt").to(device)
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 요약문 생성 및 CSV 파일 저장
summaries = []
for review in reducedData['content']:
    summary = generate_summary(review)
    summaries.append(summary)

# 요약문을 데이터프레임에 추가
reducedData['summary'] = summaries

# CSV 파일로 저장
output_file = './summarized_reviews.csv'
reducedData.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Summaries saved to {output_file}")


Using device: cuda


C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\813604558.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['content'] = reducedData['content'].fillna('')
C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\813604558.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['clean_content'] = reducedData['content'].apply(clean_text)
C:\Users\G-01\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and wil

Epoch 1


100%|██████████| 750/750 [44:05<00:00,  3.53s/it]


Loss after epoch 1: 0.5132179280991356
Summaries saved to ./summarized_reviews.csv


C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\813604558.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['summary'] = summaries


In [107]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import pandas as pd
import re

# GPU 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# T5 모델과 토크나이저 로드
model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# 리뷰 데이터를 읽어옴
file_path = '../외국음식전문점.csv'
data = pd.read_csv(file_path)

# 데이터의 첫 1000개 행만 사용
reducedData = data.iloc[:5000, :]

# 결측치 또는 NaN 데이터를 처리하여, NaN 값을 빈 문자열로 대체
reducedData['content'] = reducedData['content'].fillna('')

# 데이터 전처리: 불필요한 공백, 특수문자 제거
def clean_text(text):
    # 불필요한 공백 제거
    text = re.sub(r'\s+', ' ', text)
    # 특수문자 제거
    text = re.sub(r'[^가-힣a-zA-Z., ]', '', text)
    return text

# 모든 리뷰 텍스트 전처리
reducedData['clean_content'] = reducedData['content'].apply(clean_text)

# 데이터셋 준비
def preprocess_data(texts, max_length=512):
    input_ids = []
    attention_masks = []
    for text in texts:
        encoding = tokenizer.encode_plus(
            text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt"
        )
        input_ids.append(encoding['input_ids'])
        attention_masks.append(encoding['attention_mask'])
    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)

# 마스킹 함수
def create_mask(input_ids):
    return (input_ids != tokenizer.pad_token_id).float()

# 전처리된 데이터로부터 입력과 마스크 생성
train_inputs, train_masks = preprocess_data(reducedData['clean_content'])
train_labels, _ = preprocess_data(reducedData['content'])

# 배치 처리 및 데이터셋 준비
batch_size = 4  # 배치 크기 설정
accumulation_steps = 4  # 그래디언트 축적 단계
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

# 혼합 정밀도 학습을 위한 스케일러 설정
scaler = GradScaler()

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 학습 루프
num_epochs = 5  # 에포크 수 설정
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(train_dataloader)):
        batch_inputs, batch_masks, batch_labels = [b.to(device) for b in batch]
        
        with autocast():  # 혼합 정밀도 학습 사용
            outputs = model(input_ids=batch_inputs, attention_mask=batch_masks, labels=batch_labels)
            loss = outputs.loss / accumulation_steps  # 그래디언트 축적을 위해 손실 나누기

        scaler.scale(loss).backward()  # 스케일된 그래디언트 적용

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)  # 옵티마이저 업데이트
            scaler.update()  # 스케일러 업데이트
            optimizer.zero_grad()  # 그래디언트 초기화

        total_loss += loss.item()

    print(f"Loss after epoch {epoch + 1}: {total_loss / len(train_dataloader)}")

# 파인튜닝이 끝난 후 모델 저장
model.save_pretrained("./fine_tuned_t5")
tokenizer.save_pretrained("./fine_tuned_t5")

# 가게별로 리뷰를 묶는 함수
def group_reviews_by_store(data, store_col='store_name', review_col='content'):
    # 가게별 리뷰를 모두 합쳐서 하나의 큰 텍스트로 만듦
    grouped_data = data.groupby(store_col)[review_col].apply(lambda x: ' '.join(x)).reset_index()
    return grouped_data

# 가게별 리뷰를 묶어서 데이터를 준비
grouped_reviews = group_reviews_by_store(reducedData)

# 요약문 생성 함수
def generate_summary_for_store(store_reviews):
    input_ids = tokenizer.encode(f"summarize: {store_reviews}", return_tensors="pt").to(device)
    summary_ids = model.generate(input_ids, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 가게별 요약문 생성
summaries = []
for store_reviews in grouped_reviews['content']:
    summary = generate_summary_for_store(store_reviews)
    summaries.append(summary)

# 요약문을 데이터프레임에 추가
grouped_reviews['summary'] = summaries

# CSV 파일로 저장
output_file = './summarized_reviews_by_store.csv'
grouped_reviews.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Store summaries saved to {output_file}")


Using device: cuda


C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\1211206812.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['content'] = reducedData['content'].fillna('')
C:\Users\G-01\AppData\Local\Temp\ipykernel_2876\1211206812.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reducedData['clean_content'] = reducedData['content'].apply(clean_text)
C:\Users\G-01\AppData\Roaming\Python\Python38\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and w

Epoch 1


100%|██████████| 1250/1250 [1:14:39<00:00,  3.58s/it]


Loss after epoch 1: 0.3356435199826956
Epoch 2


...

Loss after epoch 4: 0.010958180238679052
Epoch 5


100%|██████████| 1250/1250 [49:56<00:00,  2.40s/it]


Loss after epoch 5: 0.009299165822472423


Token indices sequence length is longer than the specified maximum sequence length for this model (1636 > 512). Running this sequence through the model will result in indexing errors


Store summaries saved to ./summarized_reviews_by_store.csv
